In [ ]:
from go_no import *
from top_down import *

import slicetca
from sklearn.decomposition import PCA, NMF

import matplotlib
from matplotlib import pyplot as plt
import os
from datetime import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'none'

matplotlib.rcParams.update({'font.size': 14})

%load_ext autoreload
%autoreload 2
%matplotlib widget

device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
neuron_dimension, trial_dimension, time_dimension = 20, 30, 90
noise = 0.0
diffusion = 1.3
number_comp = 20
coef = 5
max_iter = 10**4
positive = True

colors = {'TCA':  np.array((118, 207, 183)) / 255, 'Neuron PCA': np.array((89, 158, 215)) / 255,
          'Time PCA':np.array((249, 167, 43)) / 255, 'Trial PCA': np.array((64, 185, 60)) / 255,
          'SliceTCA': (0.1, 0.1, 0.1)}


In [ ]:
t1, slice1, v1 = get_tensor_1(number_trials=trial_dimension, number_neurons=neuron_dimension, number_time=time_dimension)
t2, slice2, v2, _ = get_tensor_2(diffusion=1.3, number_neurons=neuron_dimension,number_trials=trial_dimension, number_time=time_dimension)

T = t1+t2

print(T.shape, T.mean(), T.var()) # Trial, neuron, time
T /= T.mean()

noise = 0.3
T_old = T.clone()
T += torch.randn_like(T)*noise
T = torch.relu(T)
noise_level = ((T-T_old)).mean()
print('noise level', noise_level, ((T_old-T+((T-T_old)).mean())**2).mean(), T.mean())
print(T.mean(), T.shape)

In [ ]:
extra_filename = f'trial{trial_dimension}_time{time_dimension}_neuron{neuron_dimension}_noise{noise}_positive{positive}'
now = datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
base_directory = './revisions/plots/losses/' + date_time + '__' + extra_filename
if not os.path.exists(base_directory): os.makedirs(base_directory)

In [ ]:
# ===== Neuron PCA =====
var_explained_pca_neuron = []
for i in range(1,number_comp+1):
    pca = PCA(i)
    data = T.transpose(1,2).reshape(-1,neuron_dimension).numpy()
    temp = pca.fit_transform(data)
    temp = pca.inverse_transform(temp)
    var_explained = ((data-temp)**2).mean()
    var_explained_pca_neuron.append(var_explained)
var_explained_pca_neuron = np.array(var_explained_pca_neuron)

# ===== Trial PCA =====
var_explained_pca_trial = []
for i in range(1, number_comp + 1):
    pca = PCA(i)
    data = T.transpose(0, 2).reshape(-1, trial_dimension).numpy()
    temp = pca.fit_transform(data)
    temp = pca.inverse_transform(temp)
    var_explained = ((data - temp) ** 2).mean()
    var_explained_pca_trial.append(var_explained)
var_explained_pca_trial = np.array(var_explained_pca_trial)

# ===== Time PCA =====
var_explained_pca_time = []
for i in range(1,number_comp+1):
    pca = PCA(i)
    data = T.reshape(-1,time_dimension).numpy()
    temp = pca.fit_transform(data)#
    temp = pca.inverse_transform(temp)
    var_explained = ((data-temp)**2).mean()
    var_explained_pca_time.append(var_explained)
var_explained_pca_time = np.array(var_explained_pca_time)


# ===== =====
T_cuda = T.to(device)

# ===== TCA =====
var_explained_tca = []
number_comp_tca = list(range(1,number_comp+1)) + [number_comp*2, number_comp*3, number_comp*4, number_comp*5, number_comp*10]
for i in number_comp_tca:
    components, model = slicetca.decompose(T_cuda, i, max_iter=max_iter, min_std=10**-4, positive=positive)
    var_explained_tca.append(((T_cuda-model.construct())**2).mean().item())

# ===== SliceTCA =====
var_explained_slice_tca = []
components, model = slicetca.decompose(T_cuda, [0, 1, 0], max_iter=max_iter, min_std=10**-4, positive=positive)
var_explained_slice_tca.append(((T_cuda-model.construct())**2).mean().item())
components, model = slicetca.decompose(T_cuda, [0, 1, 1], max_iter=max_iter, min_std=10**-4, positive=positive)
var_explained_slice_tca.append(((T_cuda-model.construct())**2).mean().item())

for i in range(1,number_comp-1):
    components, model = slicetca.decompose(T_cuda, [0, 1+i, 1], max_iter=max_iter, min_std=10**-4, positive=positive)
    var_explained_slice_tca.append(((T_cuda - model.construct()) ** 2).mean().item())

# ===== Store =====
var_exp = {'Neuron PCA' : var_explained_pca_neuron, 'Trial PCA' : var_explained_pca_trial,
          'Time PCA' : var_explained_pca_time, 'TCA' : var_explained_tca,
          'SliceTCA' : var_explained_slice_tca}

for k in var_exp.keys(): var_exp[k] = np.array(var_exp[k])

In [ ]:
axn = 2
y_lim = np.max(var_exp['SliceTCA']*coef)

fig = plt.figure(figsize=(4*axn+1,4), constrained_layout=True)
axs = [fig.add_subplot(1, axn, 1+i) for i in range(axn)]

ax = axs[0]
ax.set_title('Component efficiency')

ax.axhline(((T_old-T+((T-T_old)).mean())**2).mean()*coef, linestyle='--', color=(0.8, 0.8, 0.8), label='Noise floor')

for k in var_exp.keys():
    ax.plot(np.arange(number_comp)+1, var_exp[k][:number_comp]*coef, color=colors[k], label=k)

ax.set_xlabel('Number of components')
ax.set_ylabel('Error')
ax.legend()

ax.set_xlim(0, number_comp)
ax.set_ylim(-0.05, y_lim)

ax = axs[1]
ax.set_title('Parameter efficiency')

ax.axhline(((T_old-T+((T-T_old)).mean())**2).mean()*coef, linestyle='--', color=(0.8, 0.8, 0.8), label='Noise floor')

param_per_component = {'Neuron PCA' : (time_dimension*trial_dimension+neuron_dimension), 
                       'Trial PCA' : (time_dimension*neuron_dimension+trial_dimension),
                      'Time PCA' : (neuron_dimension*trial_dimension+time_dimension)}

for k in param_per_component.keys():
    ax.plot([i*param_per_component[k] for i in range(len(var_exp[k]))], var_exp[k]*coef, color=colors[k], label=k)

tca_param = [i*(time_dimension+trial_dimension+neuron_dimension) for i in number_comp_tca]
ax.plot(tca_param, var_exp['TCA']*coef, color=colors['TCA'], label='TCA')

slicetca_param = [(time_dimension*trial_dimension+neuron_dimension), (time_dimension*trial_dimension+neuron_dimension)+(trial_dimension*neuron_dimension+time_dimension)]
slicetca_param = slicetca_param + [slicetca_param[-1]+i*(time_dimension*trial_dimension+neuron_dimension) for i in range(1, number_comp + 1-2)]
ax.plot(slicetca_param, var_exp['SliceTCA']*coef, color=colors['SliceTCA'], label='SliceTCA')

ax.set_xlim(0, tca_param[-1])

xticks = [str(int(i/1000))+'k' for i in np.array(ax.get_xticks()).astype(int)]
ax.set_xticks(ax.get_xticks(), xticks)

ax.set_ylim(-0.05,y_lim)

ax.set_xlabel('Number of parameters')
ax.set_ylabel('Error')
ax.legend()

plt.savefig(base_directory+'/fig1_C_2'+str(noise)+'.pdf')